# PHASE 2: PROFIT

In [1]:
import utils.dir_utils as dir_utils
import utils.dict_utils as dict_utils
import utils.ptr_utils as ptr_utils
import utils.constants as constants 
import utils.profit as profit 
import helpers.official as official
import copy

_, input_df = dir_utils.get_data(combined=True)
_, house_input_df = dir_utils.get_data(house=True)
_, senate_input_df = dir_utils.get_data(senate=True)

### Match Purchases and Sales

In [2]:
def setup():
    purchases = set()
    sales = set()

    for _,t in input_df.iterrows():
        if ptr_utils.isvalid(t[constants.TICKER]):
            core =  (t[constants.TDATE], official.get_name(t), t[constants.TICKER])
            if t[constants.TYPE] == 'Purchase': 
                purchases.add(core)
            elif t[constants.TYPE] == 'Sale (Full)' or t[constants.TYPE] == 'Sale (Partial)':
                sales.add(core)

    return purchases, sales 


def share_diff(ticker, tdate_sale, tdate_purch):
    sale_price = profit.get_stock_price(ticker, tdate_sale)
    purch_price = profit.get_stock_price(ticker, tdate_purch)
    if purch_price and sale_price:
        return round(purch_price - sale_price, 2)
    return None 


def match(purchases, sales):
    sales_copy = copy.deepcopy(sales)

    # { (name, ticker, tdate_sale) : [(share_diff, tdate_purch), ... , ... ]  }
    possible_matches = {}
    i = 0 
    for (tdate_sale, name_sale, ticker_sale) in sales_copy: 
        i += 1 
        print(i)   
        for (tdate_purch, name_purch, ticker_purch)  in purchases:
            if name_purch == name_sale and ticker_purch == ticker_sale and ptr_utils.is_this_date_after(tdate_sale, tdate_purch):
                diff = share_diff(ticker_purch, tdate_sale, tdate_purch)
                if diff: 
                    possible_matches = dict_utils.increment_list_in_dictionary(possible_matches, (name_sale, ticker_sale, tdate_sale), (diff, tdate_purch))
            
    return possible_matches, purchases, sales 

        
p, s = setup()
print('len(sales): {}'.format(len(s)))
print('len(purchases): {}'.format(len(p)))
pm, p, s = match(p, s)
p = profit.get_instances(pm)


# { (name, ticker) : {'2017/04/05': (143.2, '2019/11/05'), '2015/07/09': (28.95, '2020/03/26'), '2017/12/07': -1, '2021/01/11': (-2.02, '2021/01/15')} }
res = {}
for i in p:
    m = profit.stableMarriage(p[i])
    res = dict_utils.increment_dictionary(res, i, m, not_math=True)    

len(sales): 8170
len(purchases): 8563
1
2
3


In [ ]:
# def get_sprice(matches):
#     for (tdate_sale, tdate_purch, name, ticker) in matches:
#             sale_price = get_stock_price(ticker, tdate_sale)
#             purch_price = get_stock_price(ticker, tdate_purch)
            
#             if not purch_price or not sale_price:
#                 pass 
#             else: 
#                 print(round(purch_price - sale_price, 2), tdate_sale, tdate_purch, name, ticker)
#                 pass 


# get_sprice(m)

                # matches.add((tdate_sale, tdate_purch, name_purch, ticker_purch))
                # sales.remove((tdate_sale, name_sale, ticker_sale))
                # purchases.remove((tdate_purch, name_purch, ticker_purch))
                # break 
                
                
                # def maximize(possible_matches):
#     # { (name, ticker, tdate_sale) : (share_diff, tdate_purch), ....  }
#     finalized = {}
    
#     for (name, ticker, tdate_sale) in possible_matches.keys():
#         best_diff = 0
#         best_date = None 
#         for (diff, tdate_purch) in possible_matches[(name, ticker, tdate_sale)]:
#             if diff > best_diff:
#                 best_diff = diff
#                 best_date = tdate_purch
                
#         finalized[(name, ticker, tdate_sale)] = (best_diff, best_date)
    
#     return finalized
        

